In [2]:
import json
import numpy as np
import pandas as pd
import plotly.express as px
from process_base import X_t_proportional

# Case 1: Proportional reinsurance

In [3]:
def get_ruin_of_instance(X_t: X_t_proportional, process_config: dict, max_time: float, seed: int):
    # update seed
    process_config.update(seed=seed)
    # make process instance
    x_t = X_t(**process_config)
    # generate process until max_time
    x_t.get_value_at_time(time=max_time)
    # get ruin probability
    for i_process in range(process_config['n'] + 1):
        # print(len(x_t.inter_arrival_times[i_process]))
        for arr_time in x_t.inter_arrival_times[i_process]:
            if arr_time <= max_time:
                value = x_t.get_value_at_time(time=arr_time)
                if value[0] < 0 or value[i_process] < 0:
                    return True
    return False


def Psi(u: list, X_t: X_t_proportional, process_config: dict, max_time: float, n_simulations: int=1000, seed_offset: int=0):
    # set initial capital
    process_config.update(u=u)
    is_ruin = [False] * n_simulations
    for i_sim in range(n_simulations):
        is_ruin[i_sim] = get_ruin_of_instance(X_t, process_config, max_time, i_sim + seed_offset)
    return sum(is_ruin) / n_simulations

In [4]:
config = json.load(open("./configs/proportional/config_1.json"))
config

{'model_settings': {'n': 1,
  'c': [0.924, 0.756],
  'alpha': [0.6],
  'poisson_rate': [0.6, 1.2],
  'claims_rate': [1.5, 1.0]},
 's': [1.0, 0.5]}

In [5]:
x_t = X_t_proportional(
    u=[10, 5],
    n=config["model_settings"]["n"],
    c=config["model_settings"]["c"],
    alpha=config["model_settings"]["alpha"],
    poisson_rate=config["model_settings"]["poisson_rate"],
    claims_rate=config["model_settings"]["claims_rate"],
    seed=42
)

In [6]:
%%time

config = {
    'n': 1,
    'c': [0.924, 0.756],
    'alpha': [0.6],
    'poisson_rate': [0.6, 1.2],
    'claims_rate': [1.5, 1.0]
}

Psi(u=[10, 5], X_t=X_t_proportional, process_config=config, max_time=1000, n_simulations=1000, seed_offset=0)

CPU times: user 10.9 s, sys: 83.4 ms, total: 11 s
Wall time: 11.2 s


0.696

In [7]:
x_t.plot_process(max_time=1000)